# C1 W4 Group 8 - Task 4


In [ ]:
from PIL import Image

from src.paths import BBDD_PATH, QST1_W4_PATH, WEEK_4_RESULTS_PATH
from tqdm import tqdm

In [ ]:
BBDD_PATH_LIST = sorted(BBDD_PATH.glob("*.jpg"))
QST1_W4_PATH_LIST = sorted(QST1_W4_PATH.glob("*.jpg"))

In [ ]:
database_image_PIL_list = [Image.open(db_img_path) for db_img_path in BBDD_PATH_LIST]  # Load once
for idx, db_img in enumerate(database_image_PIL_list):
    assert db_img.filename.endswith(f"{idx}.jpg")

In [ ]:
query_t1_image_PIL_list = [Image.open(query_img_path) for query_img_path in QST1_W4_PATH_LIST]  # Load once
for idx, query_img in enumerate(query_t1_image_PIL_list):
    assert query_img.filename.endswith(f"{idx}.jpg")

## Image Denoising

In [ ]:
from src.denoising import denoise_image


In [ ]:
denoised_query_t1_PIL_list = []
for image in query_t1_image_PIL_list:
    denoised_image = denoise_image(image)
    denoised_query_t1_PIL_list.append(denoised_image)

## Background Removal

In [ ]:
from src.background import get_paintings_cropped_images

In [ ]:
cropped_query_image_list_t1 = []
pbar = tqdm(zip(QST1_W4_PATH_LIST, denoised_query_t1_PIL_list))
for name, image in pbar:
    pbar.set_description(f"Splitting {name.stem}")
    image_list = get_paintings_cropped_images(image)
    cropped_query_image_list_t1.append(image_list)

## Retrieval

In [ ]:
from src.descriptors import ImageRetrievalSystem, SIFTDescriptor, HOGDescriptor, ORBDescriptor

In [ ]:
descriptors = {
    'SIFT': SIFTDescriptor(max_features=500),
    'HOG13': HOGDescriptor(pixels_per_cell=12, cells_per_block=4),
    'ORB': ORBDescriptor(),
}
descriptor_path = WEEK_4_RESULTS_PATH / 'Task_1' / 'descriptors'  

retrieval_system = ImageRetrievalSystem(descriptors, descriptor_path, log=False)

In [ ]:
#Set number of top similar images to retrieve
K = 10

results = {}
g_0 = {}

descriptor_name = "HOG13"

results[descriptor_name] = retrieval_system.retrieve_similar_images(descriptor_name, database_image_PIL_list, cropped_query_image_list_t1, None, K)

In [ ]:
print(results[descriptor_name])